## A note about loops on structs

In [62]:
# Let's continue to use our sum1 function!
function sum1(xs)
    s = zero(xs[1])

    for i=1:length(xs)
        s = +(s, xs[i])
    end
    s
end

sum1 (generic function with 1 method)

Let's try out a complex number type:

In [63]:
struct Cplx1
    re::Float64
    im::Float64
end

Base.:+(a::Cplx1, b::Cplx1) = Cplx1(a.re + b.re, a.im + b.im)
Base.zero(::Cplx1) = Cplx1(0.0,0.0)

In [64]:
xs = [Cplx1(rand(), rand()) for i=1:10^6]

@time sum1(xs)

  0.013903 seconds (18.87 k allocations: 1005.441 KiB)


Cplx1(500437.33469767426, 499808.80814985815)

But! We want to allow not only Float64 real and imaginary parts, we want integers, or float32 or rational numbers, whatever the user wants!

In [15]:
struct Cplx2
    re::Real
    im::Real
end

Base.:+(a::Cplx2, b::Cplx2) = Cplx2(a.re + b.re, a.im + b.im)
Base.zero(::Cplx2) = Cplx2(0.0,0.0)

In [17]:
xs = [Cplx2(rand(Float32), rand(Float32)) for i=1:10^6]

@time sum1(xs)

  0.351966 seconds (5.00 M allocations: 91.553 MiB, 60.16% gc time)


Cplx2(499805.29403316975, 499985.67185616493)

In [20]:
real2(x::Cplx2) = x.re

@code_typed real2(Cplx2(1,2))

CodeInfo(
│╻ getproperty1 1 ─ %1 = (Base.getfield)(x, :re)::Real
│   └──      return %1
) => Real

In [18]:
@code_typed sum1(xs)

CodeInfo(
│╻     getindex2 1 ──       (Base.arrayref)(true, xs, 1)::Cplx2
││╻     Type  │    %2  = %new(Main.Cplx2, 0.0, 0.0)::Cplx2
│╻     length4 │    %3  = (Base.arraylen)(xs)::Int64
││╻╷╷╷  Type  │    %4  = (Base.sle_int)(1, %3)::Bool
│││╻     unitrange_last  │          (Base.sub_int)(%3, 1)::Int64
││││    │    %6  = (Base.ifelse)(%4, %3, 0)::Int64
││╻╷╷   isempty  │    %7  = (Base.slt_int)(%6, 1)::Bool
││      └───       goto #3 if not %7
││      2 ──       goto #4
││      3 ──       goto #4
│       4 ┄─ %11 = φ (#2 => true, #3 => false)::Bool
│       │    %12 = φ (#3 => 1)::Int64
│       │    %13 = φ (#3 => 1)::Int64
│       │    %14 = (Base.not_int)(%11)::Bool
│       └───       goto #10 if not %14
│       5 ┄─ %16 = φ (#4 => %2, #9 => %20)::Cplx2
│       │    %17 = φ (#4 => %12, #9 => %26)::Int64
│       │    %18 = φ (#4 => %13, #9 => %27)::Int64
│╻     getindex5 │    %19 = (Base.arrayref)(true, xs, %17)::Cplx2
│       │    %20 = invoke Main.:+(%16::Cplx2, %19::Cplx2)::Cplx2
││

We can make a parametric type which is parameterized by the type of real and imaginary parts

In [21]:
struct Cplx3{T<:Real}
    re::T
    im::T
end


Base.:+(a::Cplx3, b::Cplx3) = Cplx3(a.re + b.re, a.im + b.im)
Base.zero(a::Cplx3) = Cplx3(zero(a.re), zero(a.im))

In [30]:
@code_typed sum1(xs)

CodeInfo(
│╻     getindex2 1 ──       (Base.arrayref)(true, xs, 1)::Cplx3{Float32}
││╻╷    Type  │    %2  = %new(Cplx3{Float32}, 0.0, 0.0)::Cplx3{Float32}
│╻     length4 │    %3  = (Base.arraylen)(xs)::Int64
││╻╷╷╷  Type  │    %4  = (Base.sle_int)(1, %3)::Bool
│││╻     unitrange_last  │          (Base.sub_int)(%3, 1)::Int64
││││    │    %6  = (Base.ifelse)(%4, %3, 0)::Int64
││╻╷╷   isempty  │    %7  = (Base.slt_int)(%6, 1)::Bool
││      └───       goto #3 if not %7
││      2 ──       goto #4
││      3 ──       goto #4
│       4 ┄─ %11 = φ (#2 => true, #3 => false)::Bool
│       │    %12 = φ (#3 => 1)::Int64
│       │    %13 = φ (#3 => 1)::Int64
│       │    %14 = (Base.not_int)(%11)::Bool
│       └───       goto #10 if not %14
│       5 ┄─ %16 = φ (#4 => 0.0f0, #9 => %22)::Float32
│       │    %17 = φ (#4 => 0.0f0, #9 => %24)::Float32
│       │    %18 = φ (#4 => %12, #9 => %31)::Int64
│       │    %19 = φ (#4 => %13, #9 => %32)::Int64
│╻     getindex5 │    %20 = (Base.arrayref)(true, x

In [26]:
xs = [Cplx3(rand(Float32), rand(Float32)) for i=1:10^6]

@time sum1(xs)

  0.001565 seconds (5 allocations: 176 bytes)


Cplx3{Float32}(499901.72f0, 499801.72f0)

In [29]:
Cplx3(1//2, 3//4) # Julia can infer the parameter

Cplx3{Rational{Int64}}(1//2, 3//4)

## Tuples

In [41]:
using Interact

┌ Info: Recompiling stale cache file /home/shashi/.julia/compiled/v1.0/Interact/XmYW4.ji for Interact [c601a237-2ae4-5e1e-952c-7a85b0c7eef1]
└ @ Base loading.jl:1187


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/e8f953a4abf5e4095b83e225fc5e11254819fa44-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/e8f953a4abf5e4095b83e225fc5e11254819fa44-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [61]:
tup1 = (1, "yes", :no)

typeof(tup1)

second(t) = t[2]


vec1 = [1, "yes", :no]

typeof(vec1)

second(t) = t[2]



hbox(@code_typed(second(tup1)), hskip(2em), @code_typed second(vec1))

(div { style=Dict("display"=>"flex","flex-direction"=>"row") }
  (div { setInnerHtml="<pre>CodeInfo(\n12 1 ─ %1 = (Base.getfield)(t, 2, true)::String\n   └──      return %1\n) => String</pre>" })
  (div { style=Dict{String,Measures.Length{U,Float64} where U}("height"=>0.0px,"width"=>2.0em) })
  (div { setInnerHtml="<pre>CodeInfo(\n12 1 ─ %1 = (Base.arrayref)(true, t, 2)::Any\n   └──      return %1\n) => Any</pre>" }))

In [45]:
vec1[2] = 2
# tup1[2] = 2

2

## Mutability

Do not **create** mutable objects in a loop!

In [46]:
mutable struct Cplx4{T<:Real}
    re::T
    im::T
end


Base.:+(a::Cplx4, b::Cplx4) = Cplx4(a.re + b.re, a.im + b.im)
Base.zero(a::Cplx4) = Cplx4(zero(a.re), zero(a.im))

In [60]:
xs = [Cplx4(rand(Float32), rand(Float32)) for i=1:10^6]

@time sum1(xs)

  0.042177 seconds (1.00 M allocations: 15.259 MiB, 86.77% gc time)


Cplx4{Float32}(499943.7f0, 500060.75f0)